In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
import zipfile
main_directory = "utility_files/dog_vs_cat_ds"


In [ ]:

test_zip = "/".join([main_directory, "test1.zip"])
train_zip = "/".join([main_directory, "train.zip"])

#unzip training data
zip_ref = zipfile.ZipFile(train_zip, 'r')
zip_ref.extractall(main_directory + '/train')
zip_ref.close()
# unzip test data

zip_ref = zipfile.ZipFile(test_zip, 'r')
zip_ref.extractall(main_directory + '/test')
zip_ref.close()

In [ ]:
import os
import shutil

# since the folder with the data contains an additional folder, we will move the images from the current folder 
# to either the cat_train or train_dog folders (are to be created)

train_dog = "/".join([main_directory, "train", "dog"])
train_cat = "/".join([main_directory, "train", "cat"])


# os.mkdir(train_dog)
# os.mkdir(train_cat)

In [ ]:
# let's move each training label to the corresponding folder

train_dir_name = "/".join([main_directory, "train"])
train_dir = os.path.join(train_dir_name + "/train")


In [ ]:

for file in os.listdir(train_dir):
    des = ""
    if "cat" in file.lower().strip():
        des = train_cat
    elif "dog" in file.lower().strip():
        des = train_dog
    
    source = "/".join([train_dir_name,"train", file])
    destination = "/".join([des, file])
    # print(source, destination, sep="\n")

    shutil.move(source, destination)


In [ ]:

# delete the old train folder
os.rmdir("/".join([main_directory, "train", "train"]))
# after diving the images into the corresponding directories, it would easier to use tensorflow's Datagenerator

In [ ]:
dog_names = os.listdir(train_dog)
cat_names = os.listdir(train_cat)

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images


In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cat, fname) 
                for fname in cat_names[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dog, fname) 
                for fname in dog_names[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


In [ ]:
model_input_shape = (150, 150, 3) # image of 150 * 150 pixels with 3 bytes for colors

# this model is taken out of the fashion_mnist_CNN notebook
import tensorflow.keras.layers as tfl

def cnn_fashion_model(input_shape, num_classes):
    # define the input
    inputs = tf.keras.Input(shape=input_shape)
    # relu activation is used extensively:
    relu = tfl.ReLU()
    # 2D convolutional layer
    f1 = 16
    size1  = (3, 3)
    conv1 = tfl.Conv2D(filters=f1, kernel_size=size1, strides=(2, 2), padding='same')
    
    pool1 = tfl.MaxPool2D((2, 2))
    f2 = 32
    conv2 = tfl.Conv2D(filters=f2, kernel_size=size1, strides=(1, 1), padding='same')
    
    # normalize the input on the the channels axis
    batnor = tfl.BatchNormalization(axis=3)
    size2 = (3, 3)
    f3 = 64
    conv3 = tfl.Conv2D(filters=f3, kernel_size=size2, strides=(2, 2), padding='same')

    f4 = 64
    conv4 = tfl.Conv2D(filters=f4, kernel_size=size2, strides=(1, 1), padding='same')

    pool2 = tfl.MaxPool2D((2, 2))

    # the neural network should be as follows:
    # con1 * 2 -> conv2 *2 -> pool1 -> conv3 * 2 -> conv4 * 2 -> pool2 -> fully connected1 -> fullyconnected2 -> softmax
    x = conv1(inputs)
    x = relu(x)

    x = conv2(inputs)
    x = relu(x)


    x = tfl.BatchNormalization(axis=3)(x)
    x = pool1(x)
    
    x = conv3(x)
    x = relu(x)

    x = conv4(x)
    x = relu(x)

    x = tfl.BatchNormalization(axis=3)(x)
    x = pool2(x)

    flatten = tfl.Flatten() 
    fc1 = tfl.Dense(128, activation='relu')
    fc2 = tfl.Dense(128, activation='relu')
    
    # the last layer depends mainly on the number of classes
    output_units = num_classes if num_classes > 2 else 1
    activation = "sigmoid" if output_units == 1 else "softmax"
    
    fc3 = tfl.Dense(output_units, activation=activation)

    x = flatten(x)
    x = fc1(x)
    x = fc2(x)
    outputs = fc3(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
our_model = cnn_fashion_model(model_input_shape, 2)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (150, 150)
train_dir = os.path.join("/".join([main_directory, "train"]))
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=image_size)     


In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']
our_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
history = our_model.fit(
            train_generator,
            epochs=15,
            verbose=2
)